### DTW Alignment code

### Variations of DTW

#### Downsampling and upsampling DTWs

In [8]:
def DTW1_downsampleQuantized(featfile1, featfile2, steps, weights, downsample, outfile = None, profile = False):
    '''Downsample longer sequence. For each position in the downsampled sequence, 
    assign the nearest feature vector (to handle fractional positions).'''
    
    F1 = np.load(featfile1) # 12 x N
    F2 = np.load(featfile2) # 12 x M

    shorter, longer = None, None
    
    # boolean to keep track of which is F1, F2
    F1_long = None
    # determine which is longer
    if F1.shape[1] >= F2.shape[1]:
        longer, shorter = F1, F2
        F1_long = True
    else:
        longer, shorter = F2, F1
        F1_long = False

    downsampled = np.zeros(shorter.shape)
    # determine factor and corresponding indices
    factor_down = longer.shape[1]/shorter.shape[1]
    idx_down = [round(factor_down*i) for i in range(shorter.shape[1])]

    for i in range(shorter.shape[1]): # for each column in new empty array, get corresponding idx to use to select value in longer
        idx = idx_down[i]
        downsampled[:,i] = longer[:,idx] # copy values over
    
    # get original variables
    if F1_long:
        F1 = downsampled
    else:
        F2 = downsampled

    # just same code as alignDTW
    return alignDTW_sub(F1, F2, downsample, steps, weights, outfile, profile)

In [10]:
def DTW1_downsampleInterpolate(featfile1, featfile2, steps, weights, downsample, outfile = None, profile = False):
    '''Same as DTW_downsample_quantized with one difference: rather than assigning the nearest feature vector, 
    handle fractional positions by doing a linear interpolation between the two bordering feature vectors'''
    
    F1 = np.load(featfile1) # 12 x N
    F2 = np.load(featfile2) # 12 x M

    shorter, longer = None, None
    
    # boolean to keep track of which is F1, F2
    F1_long = None
    # determine which is longer
    if F1.shape[1] >= F2.shape[1]:
        longer, shorter = F1, F2
        F1_long = True
    else:
        longer, shorter = F2, F1
        F1_long = False

    downsampled = np.zeros(shorter.shape)
    # determine factor and corresponding indices
    factor_down = longer.shape[1]/shorter.shape[1]
    idx_down = [factor_down*i for i in range(shorter.shape[1])]

    for i in range(shorter.shape[1]): # for each column in new empty array, get corresponding idx to use to select value in longer
        idx = idx_down[i]
        fraction1 = idx - math.floor(idx) # to use with higher number. e.g. if 1.73, then fraction1 = 0.73
        fraction2 = 1 - fraction1
        downsampled[:,i] = longer[:,math.floor(idx)]*fraction2 + longer[:,math.ceil(idx)]*fraction1 # copy values over with linear interpolation

    # get original variables
    if F1_long:
        F1 = downsampled
    else:
        F2 = downsampled

    # just same code as alignDTW
    return alignDTW_sub(F1, F2, downsample, steps, weights, outfile, profile)

In [11]:
def DTW1_upsampleQuantized(featfile1, featfile2, steps, weights, downsample, outfile = None, profile = False):
    '''For each position in the upsampled sequence, assign the nearest feature vector (to handle fractional positions)'''

    F1 = np.load(featfile1) # 12 x N
    F2 = np.load(featfile2) # 12 x M
    
    shorter, longer = None, None
    
    # boolean to keep track of which is F1, F2
    F1_long = None
    # determine which is longer
    if F1.shape[1] >= F2.shape[1]:
        longer, shorter = F1, F2
        F1_long = True
    else:
        longer, shorter = F2, F1
        F1_long = False

    upsampled = np.zeros(longer.shape)

    # determine factor and corresponding indices
    factor_up = shorter.shape[1]/longer.shape[1]
    max_idx = shorter.shape[1] - 1
    idx_up = [] # want to get indices
    for i in range(longer.shape[1]): # for i in range longer
        idx = round(factor_up*i)
        if idx > max_idx: # need to check that index not out of range
            idx -= 1
        idx_up.append(idx)

    for i in range(longer.shape[1]): # for each column in new empty array, get corresponding idx to use to select value in shorter
        idx = idx_up[i]
        upsampled[:,i] = shorter[:,idx] # copy values over
    
    # get original variables
    if F1_long:
        F2 = upsampled
    else:
        F1 = upsampled

    # just same code as alignDTW
    return alignDTW_sub(F1, F2, downsample, steps, weights, outfile, profile)

In [12]:
def DTW1_upsampleInterpolate(featfile1, featfile2, steps, weights, downsample, outfile = None, profile = False):
    '''Same as DTW_upsample_quantized with one difference: rather than assigning the nearest feature vector, 
    handle fractional positions by doing a linear interpolation between the two bordering feature vectors'''
    
    F1 = np.load(featfile1) # 12 x N
    F2 = np.load(featfile2) # 12 x M
    
    shorter, longer = None, None
    
    # boolean to keep track of which is F1, F2
    F1_long = None
    # determine which is longer
    if F1.shape[1] >= F2.shape[1]:
        longer, shorter = F1, F2
        F1_long = True
    else:
        longer, shorter = F2, F1
        F1_long = False

    upsampled = np.zeros(longer.shape)

    # determine factor and corresponding indices
    factor_up = shorter.shape[1]/longer.shape[1]
    max_idx = shorter.shape[1] - 1
    idx_up = [factor_up*i for i in range(longer.shape[1])]

    for i in range(longer.shape[1]): # for each column in new empty array, get corresponding idx to use to select value in longer
        idx = idx_up[i]
        if math.ceil(idx) > max_idx: # check for index out of range
            upsampled[:,i] = shorter[:,math.floor(idx)]
        else:
            fraction1 = idx - math.floor(idx) # to use with higher number. e.g. if 1.73, then fraction1 = 0.73
            fraction2 = 1 - fraction1
            upsampled[:,i] = shorter[:,math.floor(idx)]*fraction2 + shorter[:,math.ceil(idx)]*fraction1 # copy values over with linear interpolation
    
    # get original variables
    if F1_long:
        F2 = upsampled
    else:
        F1 = upsampled

    # just same code as alignDTW
    return alignDTW_sub(F1, F2, downsample, steps, weights, outfile, profile)

#### Adaptive weights

In [13]:
def DTW_adaptiveWeight1(featfile1, featfile2, steps, weights=None, downsample=None, outfile = None, profile = False):
    '''DTW but with a weighting scheme ensures that both axes contribute the same weighted Manhattan distance cost'''
    # calculate weights:
    F1 = np.load(featfile1) # 12 x N
    F2 = np.load(featfile2) # 12 x M

    w1, w2 = None, None # keep track of which is the longer sequence

    # calculate weight ratios
    if F1.shape[1] >= F2.shape[1]:
        Lmax, Lmin = F1.shape[1], F2.shape[1]
        w1, w2 = Lmax/Lmin, 1 # calulate ratios
    else:
        Lmax, Lmin = F2.shape[1], F1.shape[1]
        w1, w2 = 1, Lmax/Lmin # calulate ratios

    # calculate weights for each step
    weights = []
    # for step in steps, calculate weight
    for step in steps: # step is e.g. [1,1]
        wy = step[0]*w2 # step[0] is in Lmin direction
        wx = step[1]*w1
        weights.append(wy + wx)
    weights = np.array(weights)

    # just same code as alignDTW
    return alignDTW_sub(F1, F2, downsample, steps, weights, outfile, profile)


In [14]:
def DTW_adaptiveWeight2(featfile1, featfile2, steps, weights=None, downsample=None, outfile = None, profile = False):
    '''DTW but with a different weighting scheme that accounts for the length of the sequences'''
    # calculate weights w1, w2, w3
    F1 = np.load(featfile1) # 12 x N
    F2 = np.load(featfile2) # 12 x M

    w1, w2, w3 = None, None, None # keep track of which is the longer sequence

    # calculate weight ratios
    if F1.shape[1] >= F2.shape[1]:
        Lmax, Lmin = F1.shape[1], F2.shape[1]
        w1, w2 = Lmax/Lmin, 1 # calulate ratios
    else:
        Lmax, Lmin = F2.shape[1], F1.shape[1]
        w1, w2 = 1, Lmax/Lmin # calulate ratios
    w3 = 1 + Lmax/Lmin
    weights = np.array([w1, w2, w3])

    # just same code as alignDTW
    return alignDTW_sub(F1, F2, downsample, steps, weights, outfile, profile)
    

In [ ]:
# featfile1 = 'features/clean/Chopin_Op017No4/Chopin_Op017No4_Afanassiev-2001_pid9130-01.npy'
# featfile2 = 'features/clean/Chopin_Op017No4/Chopin_Op017No4_Ashkenazy-1981_pid9058-13.npy'
# steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
# weights = np.array([2,3,3])

# F1, F2, steps, weights = DTW_adaptiveWeight1(featfile1, featfile2, steps, weights, downsample, outfile = None, profile = False)

#### Selective transitions

In [15]:
def DTW_selectiveTransitions(featfile1, featfile2, steps, weights, downsample, outfile = None, profile = False):
    '''This is same as DTW except that the set of allowable transitions at each position (i,j) in the pairwise cost matrix may be different
Let the max desired time warp factor be WarpMax (an integer)
If i%WarpMax == 0, then remove (1,0) transition from that cell
If j%WarpMax == 0, then remove (0,1) transition from that cell
Otherwise, keep the transitions (0,1), (1,0), (1,1) with weights 1, 1, 2
See example figure below for WarpMax = 3
'''